<h1 style="color:#D4AF37">Exploratory Analysis of an Electrical Outage Dataset</h1>

<h2>Introduction:</h2>

While exploring datasets related to the energy sector, I came across a dataset related to electrical outages labeled "AEMR" which could be some sort of regulator. Though I had no prior knowledge about this specific company or its operations, I was intrigued to dive deep and uncover insights from the data. This notebook documents my exploratory journey.

<h2>Objective:</h2>
Intrigued by the patterns and trends within the data, I set two primary objectives for my exploratory analysis:

<b>A) Investigate Energy Stability and Market Outages:</b>
I wanted to understand the frequency, duration, and potential causes of these outages. By doing so, I aimed to provide insights into how energy stability might be impacted by these disruptions.

<b>B) Examine Energy Losses and Market Reliability:</b>
My goal was to quantify the energy losses during these outages and to assess their implications on the overall reliability of the energy market. This could help in understanding the broader consequences of such events and the potential areas of intervention.

<h2>Dataset:</h2>
The dataset consists of various outage events, detailing the facility involved, duration, cause, and the power lost.
Since the dataset was discovered without accompanying documentation, part of this exploration will involve understanding its structure and content.

The four types of outages categorized are:

● Consequential

● Forced 

● Opportunistic 

● Planned 


# Let's Connect

I'm always eager to take on new challenges and to collaborate with fellow data enthusiasts. Whether you're a recruiter looking for talent, a peer wanting to collaborate, or just someone curious about data, I'd love to connect.
  
  [GitHub](https://github.com/mstripling/Data-Analyst/)

  [LinkedIn](https://www.linkedin.com/in/miles-stripling)
  
  [Tableau](https://public.tableau.com/app/profile/miles.stripling?authMode=activationSuccess)

  miles.stripling@gmail.com


<h3>Initial configuration of the notebook settings.</h3>

In [3]:
%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///AEMR.db")
%load_ext sql
%sql sqlite:///AEMR.db

In [4]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});

<IPython.core.display.Javascript object>

<h2>Examine table schema and get a baseline</h2>

In [5]:
%%sql
select * FROM AEMR_Outage_Table
LIMIT 10

 * sqlite:///AEMR.db
Done.


EventID,Start_Time,End_Time,Year,Month,Facility_Code,Participant_Code,Status,Outage_Reason,Energy_Lost_MW,Description_Of_Outage
1,2017-12-28 06:00,2017-12-28 10:00,2017,12,DNHR_DENMARK_WF1,DNHR,Approved,Consequential,1.44,a network outage in Denmark caused the windfarm to trip at 6:26 am
2,2017-12-26 09:00,2017-12-27 00:00,2017,12,COLLGAR_WF1,COLLGAR,Approved,Forced,30.0,Forced Outage - BS on Overhead line pole S-81 - CG10/11
3,2017-12-31 09:00,2017-12-31 09:00,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,5.702999999999999,Under generation - ambient conditions
4,2017-12-31 08:30,2017-12-31 08:30,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,5.085,Under generation - ambient conditions
5,2017-12-31 08:00,2017-12-31 08:00,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,5.02,Under generation - ambient conditions
6,2017-12-30 23:30,2017-12-30 23:30,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,5.79,Under generation - ambient conditions
7,2017-12-30 23:00,2017-12-30 23:00,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,5.412000000000001,Under generation - ambient conditions
8,2017-12-30 22:30,2017-12-30 22:30,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,4.735,Under generation - ambient conditions
9,2017-12-29 23:30,2017-12-29 23:30,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,4.245,Under generation - ambient conditions
10,2017-12-29 23:00,2017-12-29 23:00,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,3.9619999999999997,Under generation - ambient conditions


## <span style="color:#D4AF37">Part I. Energy Stability & Market Outages</span>

In my exploration of the dataset, I've recognized energy stability as a pivotal theme. To guarantee energy security and reliability, I've decided to address the following:

- I want to determine the most prevalent types of outages and their typical durations.
- I aim to quantify the regularity of these outages.
- I'm curious to identify if certain energy providers experience more outages than their counterparts, potentially hinting at reliability issues.

<span style="color:Maroon">**Note:**
For the sake of accuracy in this analysis, I've chosen to focus solely on the Outages with the status labeled as "Approved". This means I'll consistently filter the data using the condition `Where Status = Approved`, ensuring that canceled or unapproved outages won't influence the results. I'll also be filtering the top 10 results from each query for readability. </span>


The first query will be to count the number of valid (`Status = Approved`) outage events sorted by the specific outage reason over the 2016 & 2017 periods.

In [6]:
%%sql
-- Count the number of each type of outage by month
SELECT COUNT(EventID) as Total_Number_outages, Outage_Reason, Month, Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Outage_Reason, Month
ORDER BY Outage_Reason
LIMIT 10;


 * sqlite:///AEMR.db
Done.


Total_Number_outages,Outage_Reason,Month,Year
36,Consequential,1,2017
50,Consequential,2,2017
26,Consequential,3,2017
12,Consequential,4,2017
41,Consequential,5,2017
20,Consequential,6,2017
24,Consequential,7,2017
8,Consequential,8,2017
11,Consequential,9,2017
26,Consequential,10,2017


Forced outages are far more frequent than any other.
These are also the least desirable type of outage.

<h3 style="color:#D4AF37"> Question Two </h3>



<h3 style="color:#D4AF37"> Step II: Diving Deeper into Outage Patterns </h3>

Given the foundational insights from the initial exploration, I wanted to delve deeper into the nuances of the outages. Specifically, I aimed to:
<b>
- Understand the monthly distribution of all outage types (Forced, Consequential, Scheduled, Opportunistic) for 2016 and 2017, ensuring I focus on approved outages.
- Examine any specific outage type that might be on the rise when comparing 2016 to 2017.
</b>

<h4 style="color:Teal"> Please write your SQL in the code window below </h4>

In [7]:
%%sql
-- Looking at 'Approved' outages, find the total count by month
with cte as (
SELECT Year, Month, Outage_Reason
FROM AEMR_Outage_Table
WHERE Status = 'Approved' 
)
SELECT Year, Month, Outage_Reason, COUNT(*) as Total_Number_Outages
FROM cte
GROUP BY Year, Month, Outage_Reason
ORDER BY Year, Month, Outage_Reason
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Year,Month,Outage_Reason,Total_Number_Outages
2016,1,Consequential,24
2016,1,Forced,134
2016,1,Opportunistic Maintenance (Planned),9
2016,1,Scheduled (Planned),24
2016,2,Consequential,23
2016,2,Forced,149
2016,2,Opportunistic Maintenance (Planned),12
2016,2,Scheduled (Planned),43
2016,3,Consequential,7
2016,3,Forced,94


Having identified that a particular outage type *(forced)* was rapidly increasing from 2016 to 2017, I felt compelled to go beyond just frequency. I believed that the <b> duration </b> of an outage could offer a more comprehensive view of its severity.

Short outages might not be alarming, but prolonged ones could jeopardize energy supplies. Therefore, pinpointing any energy providers with problematic patterns seemed like a logical next step.

<h3 style="color:#D4AF37"> Step III: Duration and Reliability Analysis </h3>

I decided to probe further by:

* Calculating the total number of outage events and their average durations (in days) for each participant code and outage type across 2016 and 2017. Only approved outages were considered for this analysis.
* This would help in understanding the reliability of different energy providers based on the duration and frequency of their outages.

In [8]:
%%sql
SELECT 
    Participant_Code,
    Outage_Reason,
    Year,
    COUNT(EventID) AS Total_Number_Outage_Events,
    ROUND (AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code, Outage_Reason
ORDER BY Average_Outage_Duration_In_Days DESC, Outage_Reason, Year
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days
WGUTD,Scheduled (Planned),2017,1,10.58
MELK,Scheduled (Planned),2017,155,5.64
KORL,Scheduled (Planned),2017,36,5.37
COLLGAR,Scheduled (Planned),2017,21,5.17
ENRG,Scheduled (Planned),2017,66,4.91
EUCT,Forced,2017,14,4.64
GW,Scheduled (Planned),2017,71,3.78
MUND,Scheduled (Planned),2017,25,3.01
WGUTD,Forced,2017,10,2.76
PJRH,Scheduled (Planned),2017,73,2.48


The picture is starting to become clearer now. I've pinpointed participants with frequent outages and those with prolonged offline durations.

With these insights in hand, I believe it's crucial to categorize our participants based on uptime reliability metrics.

Here's how I've decided to classify the participants:

* High Risk: Participants who, on average, are unavailable for more than 24 hours.
* Medium Risk: Participants with an average unavailability between 12 and 24 hours.
* Low Risk: Participants who are typically offline for less than 12 hours.

<h3 style="color:#D4AF37">Step IV: Participant Reliability Classification</h3>

To further refine my analysis, I've decided to classify each participant based on the risk criteria I previously established. This will provide a clearer perspective on the reliability of each participant.

Here's what I aim to achieve:

* Using the risk criteria (High Risk, Medium Risk, Low Risk), I want to craft an SQL Statement that assigns a "Risk_Classification" to each participant code. This classification should be based on their Average Outage Duration Time.
* It's essential to consider only valid outages (i.e., Where status = approved) across all types (Forced, Consequential, Scheduled, Opportunistic) and for all participant codes from 2016 to 2017.
* The final results should be ordered by "Average Duration Time In Days" in descending order.

In [9]:
%%sql
WITH cte AS (
SELECT
    Participant_Code,
    Outage_Reason,
    Year,
    COUNT(EventID) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code, Outage_Reason, Year
)
SELECT
    Participant_Code,
    Outage_Reason,
    Year,
    Total_Number_Outage_Events,
    Average_Outage_Duration_In_Days,
    CASE
        WHEN Average_Outage_Duration_In_Days < 0.5 THEN 'Low Risk'
        WHEN Average_Outage_Duration_In_Days < 1 THEN 'Medium Risk'
        ELSE 'High Risk' END AS Risk_Classification
FROM cte
ORDER BY Average_Outage_Duration_In_Days DESC, Outage_Reason, Year
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
WGUTD,Scheduled (Planned),2017,1,10.58,High Risk
KORL,Scheduled (Planned),2016,20,6.96,High Risk
MELK,Scheduled (Planned),2017,70,6.89,High Risk
COLLGAR,Scheduled (Planned),2017,9,6.36,High Risk
EUCT,Forced,2016,11,5.9,High Risk
ENRG,Scheduled (Planned),2017,37,4.96,High Risk
ENRG,Scheduled (Planned),2016,29,4.85,High Risk
MELK,Scheduled (Planned),2016,85,4.61,High Risk
GW,Scheduled (Planned),2016,45,4.43,High Risk
COLLGAR,Scheduled (Planned),2016,12,4.28,High Risk


Having classified our participants into High, Medium, and Low Risk categories, I felt the need to delve even further.

I began to question: Is it justifiable to consider Consequential, Opportunistic, or Planned outages when determining risk? Given that Forced Outages are the unplanned disruptions posing the most significant threat to the electricity grid's security, perhaps it's more apt to anchor our Risk Category solely on them.

With this perspective, I've decided to refine our risk classification. I'll introduce two more criteria, focusing on the Total Number of Outage Events and the specific Outage Type.

Here's the updated classification I've conceptualized:

* High Risk: Participants who, on average, are unavailable for more than 24 hours or have more than 20 outage events.
* Medium Risk: Participants with an average unavailability between 12 and 24 hours or have outage events between 10 and 20.
* Low Risk: Participants who are typically offline for less than 12 hours or have fewer than 10 outage events.
* If the outage type isn't forced, then the risk classification is N/A.

<h3 style="color:#D4AF37">Step V: Further Classification</h3>

In [10]:
%%sql
WITH cte AS (
SELECT
    Participant_Code,
    Outage_Reason,
    Year,
    COUNT(EventID) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code, Outage_Reason, Year
)
SELECT
    Participant_Code,
    Outage_Reason,
    Year,
    Total_Number_Outage_Events,
    Average_Outage_Duration_In_Days,
    CASE
        WHEN Average_Outage_Duration_In_Days < 0.5 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events < 10 AND Outage_Reason = 'Forced' THEN 'Low Risk'
        WHEN Average_Outage_Duration_In_Days <= 1 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events <= 20 AND Total_Number_Outage_Events >= 10 AND Outage_Reason = 'Forced' THEN 'Medium Risk'
        WHEN Average_Outage_Duration_In_Days > 1 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events > 20 AND Outage_Reason = 'Forced' THEN 'High Risk'
        ELSE 'N/A' END AS Risk_Classification
FROM cte
ORDER BY Average_Outage_Duration_In_Days DESC, Outage_Reason, Year
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
WGUTD,Scheduled (Planned),2017,1,10.58,N/A
KORL,Scheduled (Planned),2016,20,6.96,N/A
MELK,Scheduled (Planned),2017,70,6.89,N/A
COLLGAR,Scheduled (Planned),2017,9,6.36,N/A
EUCT,Forced,2016,11,5.9,Medium Risk
ENRG,Scheduled (Planned),2017,37,4.96,N/A
ENRG,Scheduled (Planned),2016,29,4.85,N/A
MELK,Scheduled (Planned),2016,85,4.61,N/A
GW,Scheduled (Planned),2016,45,4.43,N/A
COLLGAR,Scheduled (Planned),2016,12,4.28,N/A


<h3 style="color:#D4AF37">  Part II. Energy Losses & Market Reliability  </h3>

As energy providers commit energy to the market, they're essentially making a promise: “We're contractually bound to supply X amount of energy.” But challenges arise when outages, especially those that are forced, disrupt these commitments. When energy providers are unexpectedly offline, it doesn't just affect their operations — <b style="color:Maroon">if multiple providers are forced offline simultaneously, it could jeopardize the entire energy security, presenting a challenge that must be addressed.</b>

With this in mind, I've decided to delve into:

* The proportion of Forced Outage(s) among all outage types for 2016 and 2017.
* The average duration of forced outages in these years, and whether there's been a noticeable increase in their duration.
* Identifying the energy providers that frequently face forced outages.

<b>Let's dive into these inquiries below.</b>

<h3 style="color:#D4AF37"> Analysis Step VI: Proportion of Forced Outages </h3>

<b> To further comprehend the dynamics of outages, I've set out to determine the proportion of Forced Outages over the 2016 - 2017 period. Additionally, I aim to identify any noticeable trends or increases in specific Outage Types during this timeframe. </b>

In [11]:
%%sql
SELECT 
    Year,
    COUNT(DISTINCT EventID) AS Total_Number_Outages,
    COUNT (CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE NULL END) AS Total_Number_Forced_Outage_Events,
    ROUND(ROUND(COUNT (CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE NULL END), 2)/
            ROUND(COUNT(DISTINCT EventID), 2)*100,2) AS Pct_Outage_Forced
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Year

 * sqlite:///AEMR.db
Done.


Year,Total_Number_Outages,Total_Number_Forced_Outage_Events,Pct_Outage_Forced
2016,1931,1264,65.46
2017,2171,1622,74.71


From this data, it's evident that Forced Outages present a significant challenge. Beyond the financial implications due to their unplanned nature, there seems to be a subset of Energy Participants consistently facing a high frequency of outages.

So, the question arises: <b>How can we address this?</b>

To get a comprehensive understanding, I've decided to segment our analysis into Macro and Micro perspectives. While the total duration indicates the cumulative time a participant is offline or experiencing energy loss, it doesn't illuminate the frequency of these occurrences. A few prolonged outages could skew our perception.

By analyzing the average, I aim to gauge the typical magnitude of these outages spread throughout the year.

Let's take a look.

<h3 style="color:#D4AF37"> Analysis Step VII: Outages by Participant and Facility </h3>

To gain a more granular view of the outages, I've decided to quantify the total number of outages, total duration, and total energy lost for each outage. I'll break this down by participant and facility code to identify any individuals.

In [12]:
%%sql
SELECT
    COUNT(EventID) as Total_Number_Outages,
    ROUND(SUM(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) as Total_Duration_In_Days,
    ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
    Outage_Reason, Participant_Code, Facility_Code, Year
from AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_code, Facility_Code, Outage_Reason, Year
ORDER BY Total_Duration_In_Days DESC
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
70,482.58,7499.28,Scheduled (Planned),MELK,MELK_G7,2017
177,404.15,10285.4,Forced,MELK,MELK_G7,2017
85,392.25,9668.79,Scheduled (Planned),MELK,MELK_G7,2016
227,240.69,19326.56,Forced,GW,BW1_GREENWATERS_G2,2017
45,199.4,6450.0,Scheduled (Planned),GW,BW1_GREENWATERS_G2,2016
37,183.58,516.92,Scheduled (Planned),ENRG,ENRG_KALGOORLIE_GT3,2017
29,140.6,445.6,Scheduled (Planned),ENRG,ENRG_KALGOORLIE_GT3,2016
20,139.25,1685.6,Scheduled (Planned),KORL,KORL_GT3,2016
157,129.6,13771.07,Forced,MELK,MELK_G7,2016
317,120.6,15751.38,Forced,GW,BW1_GREENWATERS_G2,2016


<h3 style="color:#D4AF37"> Analysis Step VIII: Averages for Forced Outgages</h3>

To better understand the impact of forced outages, I'm keen on determining the average duration and average energy lost. This will be analyzed for each participant and facility.

In [13]:
%%sql
SELECT
    ROUND(AVG(ABS(JULIANDAY(End_Time)-JULIANDAY(Start_Time))),2) AS Avg_Duration_In_Days,
    ROUND(AVG(Energy_Lost_MW),2) AS Avg_Energy_Lost,
    Outage_Reason, Participant_Code, Facility_Code, Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Outage_Reason, Participant_code, Facility_Code, Year
ORDER BY Avg_Energy_Lost DESC, Year DESC
LIMIT 5;

 * sqlite:///AEMR.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
1.11,149.0,Forced,COLLGAR,COLLGAR_WF1,2016
0.04,141.21,Forced,PMC,PMC_AG,2017
0.49,131.78,Forced,PMC,PMC_AG,2016
0.83,87.71,Forced,MELK,MELK_G7,2016
1.06,85.14,Forced,GW,BW1_GREENWATERS_G2,2017


<h3 style="color:#D4AF37"> Analysis Step IX: Energy Impact of Forced Outages </h3>

<b> I've chosen to delve deeper into the energy implications of forced outages. Specifically, I aim to determine the Average Energy Lost and Total Energy Lost for each facility and participant across the 2016 and 2017 periods, focusing on instances where the outage reason is labeled as Forced. Additionally, I'll compute the <u>percentage</u> of energy lost attributed to forced outages for each facility.
    
With this analysis, I seek to identify the participants most responsible for the energy losses due to Forced Outages.

In [14]:
%%sql
SELECT
    ROUND(AVG(Energy_Lost_MW), 2) AS Avg_Energy_Lost,
    ROUND(SUM(Energy_Lost_MW), 2) AS Total_Energy_Lost,
    ROUND(SUM(Energy_Lost_MW)/
          (SELECT SUM(Energy_Lost_MW) FROM AEMR_Outage_Table WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100, 2) as Pct_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY
    Facility_Code,
    Participant_Code,
    Year,
    Outage_Reason
ORDER BY Total_Energy_Lost DESC
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,14.21,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19326.56,12.69,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.38,10.34,Forced,GW,BW1_GREENWATERS_G2,2016
87.71,13771.07,9.04,Forced,MELK,MELK_G7,2016
51.42,10696.28,7.02,Forced,AURICON,AURICON_PNJ_U1,2016
58.11,10285.4,6.75,Forced,MELK,MELK_G7,2017
131.78,9093.09,5.97,Forced,PMC,PMC_AG,2016
72.61,5881.52,3.86,Forced,PJRH,PJRH_GT11,2016
141.21,5648.44,3.71,Forced,PMC,PMC_AG,2017
29.17,5016.67,3.29,Forced,TRMOS,TIWEST_COG1,2017


<h3 style="color:#D4AF37"> Analysis Step X: Deep Dive into Top Participants' Energy Loss </h3>

<b>Having spotlighted the top three participants responsible for the most significant energy losses — `GW`, `MELK`, and `Auricon` — I've decided to closely examine the Total Energy Lost associated with each of these Participant. Furthermore, I aim to pinpoint the description of the outage linked to the most substantial energy loss for each participant and facility. 

<u>As a final step, I'll compute the percentage of Energy Loss attributed to these specific outage reasons.</u> </b>

In [15]:
%%sql
SELECT *
From(
SELECT
    Participant_Code, Facility_Code, Description_Of_Outage,
    ROUND(SUM(Energy_Lost_MW), 2) AS Total_Energy_Lost,
    ROUND(SUM(Energy_Lost_MW)/(SELECT SUM(Energy_Lost_MW) FROM AEMR_Outage_Table WHERE Status = 'Approved' AND Outage_Reason = 'Forced') *100,2) as Pct_Energy_Lost,
    RANK() OVER (PARTITION BY Participant_Code, Facility_Code
                 ORDER BY SUM(Energy_Lost_MW) DESC) AS Rank
FROM AEMR_Outage_Table
WHERE 
    Status = 'Approved'
    AND Outage_Reason = 'Forced'
    AND Participant_Code IN ('MELK', 'GW', 'AURICON')
GROUP BY
    Participant_Code,
    Facility_Code,
    Description_Of_Outage
ORDER BY 
    Participant_Code,
    Total_Energy_Lost DESC
)
WHERE Rank = 1;

 * sqlite:///AEMR.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Pct_Energy_Lost,Rank
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,3.96,1
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28687.54,18.84,1
MELK,MELK_G7,Safety Issues,1100.0,0.72,1


# Analysis Summary:

While exploring some datasets, I stumbled upon this intriguing collection of information related to energy outages. Without any accompanying documentation, I embarked on a journey to unravel its secrets. Here's a glimpse of what I've inferred:

* Energy Stability & Market Outages: The data paints a picture of diverse energy outages with forced outages standing out prominently. Their durations, frequencies, and association with specific providers shed light on potential challenges in ensuring energy stability.

* Outages by Participant and Facility: By diving deeper, it became apparent that forced outages have been on the rise over the recent years, hinting at underlying issues in the energy market.

* Averages for Forced Outages: The average durations and energy losses due to these outages emphasize their significant impact on the overall energy landscape.

* Energy Impact of Forced Outages: The deeper layers of the dataset unveiled which participants bore the brunt of these outages, providing a roadmap of where interventions might be most needed.

* Deep Dive into Top Participants' Energy Loss: An in-depth exploration of the top three participants presented a clearer picture of the main culprits behind significant energy losses.

From this seemingly obscure dataset, I've managed to extract insights that could be pivotal for understanding and enhancing energy security. It's a testament to the power of data analysis and the stories that numbers can tell, even when they initially appear without context.

Thank you for joining me in this unexpected exploration. Your time and curiosity are truly valued.

# Let's Connect

I'm always eager to take on new challenges and to collaborate with fellow data enthusiasts. Whether you're a recruiter looking for talent, a peer wanting to collaborate, or just someone curious about data, I'd love to connect.
  
  [GitHub](https://github.com/mstripling/Data-Analyst/)

  [LinkedIn](https://www.linkedin.com/in/miles-stripling)
  
  [Tableau](https://public.tableau.com/app/profile/miles.stripling?authMode=activationSuccess)

  miles.stripling@gmail.com
